In [1]:
# import library
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import re
import json
import ast
import warnings
warnings.filterwarnings("ignore")

# Data understanding

`Kost Data (Raw).csv` provides information about scraped boarding house data in Indonesia from https://infokost.id/. Here are some explanations about how to read the scraped data.

- If `tipe_kamar == 'yes'`, then there are some additional information about the room type (e.g. room class-A, class-B, etc). Conversely, If `tipe_kamar == 'no'`, then there aren't any additional information about the room type.
- Each value of `tipe_kamar (nama_kamar)`, `tipe_kamar (luas_kamar)`, `tipe_kamar (harga_kamar)`, `tipe_kamar (spesifikasi_kamar)`, and `tipe_kamar (fasilitas_kamar)` are represented as **list of lists** if `tipe_kamar == 'yes'`, whereas if `tipe_kamar == 'no'` then the values are represented as **empty list**.
- `luas_kamar` will be `NaN` if `tipe_kamar == 'yes'` because the room areas will be described on `tipe_kamar (luas_kamar)` column.
- Empty list values on `fasilitas_gedung` and `sekitar_gedung` interpret as missing values because there aren't any such information on particular kost website.

In [2]:
kost_raw = pd.read_csv('dataset/Kost Data (Raw).csv')
print(kost_raw.shape)

(3434, 17)


In [3]:
kost_raw

,link_kost,nama_kost,alamat_kost_1,alamat_kost_2,latitude,longitude,jenis_kost,luas_kamar,fasilitas_gedung,sekitar_gedung,harga_mulai,tipe_kamar,tipe_kamar (nama_kamar),tipe_kamar (luas_kamar),tipe_kamar (harga_kamar),tipe_kamar (spesifikasi_kamar),tipe_kamar (fasilitas_kamar)
0,https://infokost.id/listing/ruoptions-caraka-i...,Caraka Inn Pesanggrahan,"Jl. Pesanggrahan V No.25, RT.8/RW.7, South Pet...","PETUKANGAN SELATAN, PESANGGRAHAN, ...",-6.2400225,106.754748,Kost Campur,NaN,"[' Area parkir', ' CCTV', ' Ruang santai / rua...","['\xa0\xa0Supermarket', '\xa0\xa0Sekolah', '\x...",Rp 1.250.000/bulan,yes,"[' Pocket Single', ' Pocket Double']","['Luas kamar: 6.5 -- 7.5m', 'Luas kamar: 6.5 -...","[' Rp 1.250.000', ' ', ' Rp 1.250.000', ' ']","[['1 Tempat tidur (100x200) 1 orang', 'Tidak t...","[['AC', 'Lemari', 'TV'], ['AC', 'Lemari', 'TV']]"
1,https://infokost.id/listing/kost-pondok-latifa...,Kost Pondok Latifa Coblong Bandung,jl titimplik no18 rt07 rw09 kelurahan sadang s...,"GEGERKALONG, SUKASARI, KOT...",-6.896961223,107.621984,Kost Putri,Luas Kamar: 8--10 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 700.000/bulan,no,[],[],[],[],[]
2,https://infokost.id/listing/kost-the-loft-cobl...,Kost The Loft Coblong Bandung,jln. cisitu baru no. 60b. dago. coblong. bandung,"BATUNUNGGAL, BANDUNG KIDUL, ...",-6.88379416,107.612552,Kost Putra,Luas Kamar: 13 -- 15 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 2.100.000/bulan,no,[],[],[],[],[]
3,https://infokost.id/listing/kost-perum-alamand...,Kost Perum Alamanda Coblong Bandung,"Jl. Tubagus Ismail Raya No.60, Sekeloa, Kecama...","SADANG SERANG, COBLONG, KO...",-6.885143026,107.622391,Kost Campur,Luas Kamar: 10 -- 12 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 1.300.000/bulan,no,[],[],[],[],[]
4,https://infokost.id/listing/kost-nijisi-house-...,Kost Nijisi House Tipe A Coblong Bandung,"Jl. Kanayakan Baru 23, Dago, Coblong, Bandun","SARIJADI, SUKASARI, KOTA B...",-6.878314919,107.617798,Kost Putri,Luas Kamar: 8--10 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 1.750.000/bulan,no,[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3429,https://infokost.id/listing/kost-ibu-sumarni-b...,Kost Ibu Sumarni Bandara Husein Nurtanio Bandung,"Jalan Pajajaran Dalam No.198/72, RT.01/RW.03, ...","LEBAK GEDE, COBLONG, KOTA ...",-6.906038881,107.586025,Kost Campur,Luas Kamar: 8--10 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 900.000/bulan,no,[],[],[],[],[]
3430,https://infokost.id/listing/kost-syifa-sukajad...,Kost syifa Sukajadi Bandung,Jln karang tineng dalam gg H gojali rt 05/05 n...,"ARJUNA, CICENDO, KOTA BAND...",-6.8920454,107.592725,Kost Campur,Luas Kamar: 15--20 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 700.000/bulan,no,[],[],[],[],[]
3431,https://infokost.id/listing/kost-griya-narayan...,Kost Griya Narayana Prapanca Tipe A Mantrijero...,"Jl Prapanca no.24 Rt 57 Gedongkiwo, Mantrijero...","GEDONGKIWO, MANTRIJERON, K...",-7.8172626,110.353949,Kost Putri,Luas Kamar: 10 -- 12 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 850.000/bulan,no,[],[],[],[],[]
3432,https://infokost.id/listing/kost-amira25-cicen...,Kost Amira25 Cicendo Bandung,"jalan junjunan dalam 1, No. 25, kecamatan Cice...","SUKAGALIH, SUKAJADI, KOTA ...",-6.895719968,107.587896,Kost Campur,Luas Kamar: 15--20 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 850.000/bulan,no,[],[],[],[],[]


# Data wrangling

## Clean `harga_mulai` column

In [4]:
def clean_harga_mulai(x):
    """clean 'harga_mulai' column and get the mean values if there are range of prices"""
    
    x = x.replace('.', '').strip('Rp / bulan').split('-')
    if len(x) == 2:
        return (float(x[1]) + float(x[0]))/2
    else:
        return float(x[0])

In [5]:
# before
display(kost_raw['harga_mulai'].head())

0    Rp 1.250.000/bulan
1      Rp 700.000/bulan
2    Rp 2.100.000/bulan
3    Rp 1.300.000/bulan
4    Rp 1.750.000/bulan
Name: harga_mulai, dtype: object

In [6]:
# after
kost_raw['harga_mulai'] = kost_raw['harga_mulai'].apply(lambda x: clean_harga_mulai(x))
display(kost_raw['harga_mulai'].head())

0    1250000.0
1     700000.0
2    2100000.0
3    1300000.0
4    1750000.0
Name: harga_mulai, dtype: float64

## Clean `luas_kamar` column

In [7]:
def clean_luas_kamar(x):
    """clean 'luas_kamar' column and get the mean values if there are range of room areas"""
    
    x = x.strip('Luas Kamar:').replace('≤','').replace('-', ' ').replace(',', '.').split()
    if len(x) == 2:
        return (float(x[1]) + float(x[0]))/2
    else:
        return float(x[0])

In [8]:
# before
display(kost_raw['luas_kamar'].head())

0                       NaN
1       Luas Kamar: 8--10 m
2    Luas Kamar: 13 -- 15 m
3    Luas Kamar: 10 -- 12 m
4       Luas Kamar: 8--10 m
Name: luas_kamar, dtype: object

In [9]:
# after
kost_raw['luas_kamar'] = kost_raw['luas_kamar'].fillna('0').apply(lambda x: clean_luas_kamar(x))
display(kost_raw['luas_kamar'].head())

0     0.0
1     9.0
2    14.0
3    11.0
4     9.0
Name: luas_kamar, dtype: float64

## One-hot encode `fasilitas_gedung` column

In [10]:
def clean_fasilitas_gedung(x):
    temp_list = ast.literal_eval(x)
    x = [val.strip().lower() for val in temp_list]
    return x

In [11]:
# create list of lists from 'fasilitas_gedung'
list_of_lists_fasilitas_gedung = kost_raw['fasilitas_gedung'].apply(lambda x: clean_fasilitas_gedung(x)).tolist()
display(list_of_lists_fasilitas_gedung[:5])

# one-hot encode each of record from 'list_of_lists_fasilitas_gedung'
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(list_of_lists_fasilitas_gedung).transform(list_of_lists_fasilitas_gedung)
df_fasilitas_gedung = pd.DataFrame(te_ary, columns = ['fasilitas_gedung_' + col for col in te.columns_]).replace({True:1, False:0})
display(df_fasilitas_gedung.head())

print('total of unique facilities:', len(df_fasilitas_gedung.columns.tolist()))

[['area parkir', 'cctv', 'ruang santai / ruang tamu', 'wifi'], [], [], [], []]

,fasilitas_gedung_ac,fasilitas_gedung_area makan,fasilitas_gedung_area parkir,fasilitas_gedung_cctv,fasilitas_gedung_cleaning service,fasilitas_gedung_dapur,fasilitas_gedung_dispenser,fasilitas_gedung_gym,fasilitas_gedung_jendela menghadap luar,fasilitas_gedung_kolam renang,fasilitas_gedung_kompor,fasilitas_gedung_kulkas,fasilitas_gedung_laundry,fasilitas_gedung_lift,fasilitas_gedung_meja kompor,fasilitas_gedung_mesin cuci,fasilitas_gedung_microwave,fasilitas_gedung_rooftop,fasilitas_gedung_ruang santai / ruang tamu,fasilitas_gedung_tempat jemur pakaian,fasilitas_gedung_wifi
0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


total of unique facilities: 21


In [12]:
# concatenate 'df_fasilitas_gedung' with 'kost_raw'

kost_raw = pd.concat([kost_raw, df_fasilitas_gedung], axis = 1)
print(kost_raw.shape)

(3434, 38)


## One-hot encode `sekitar_gedung` column

In these section, there are two steps to encode `sekitar_gedung`:
- First, encode all the entities in `sekitar_gedung`. These entities will include raw building objects such as `"universitas trisakti"`, `"universitas tarumanegara"`, `"rs sumber waras"`, `"rumah sakit"`, and many more.
- Second, aggregate the raw entities into a single unique name. For example, we can aggregate `"universitas trisakti"` and `"universitas tarumanegara"` into a single entity name called as `"kampus"`.

The main building objects we will create in which the building is near the kos-kosan are `'mall'`, `'supermarket'`, `'kampus'`, `'akses kesehatan'`, `'sekolah'`, and `'akses transportasi'`.

In [13]:
def clean_sekitar_gedung(x):
    temp_list = ast.literal_eval(x)
    x = [val.strip().strip('\xa0').lower() for val in temp_list]
    return x
    
def aggregate_sekitar_gedung(regex_building):
    selected_columns = [val for val in df_sekitar_gedung.columns.tolist() if re.findall(regex_building, val)]
    item_columns = df_sekitar_gedung[selected_columns]

    # return category of aggregated items
    return item_columns.sum(axis = 1) >= 1, selected_columns

In [14]:
# create list of lists from 'sekitar_gedung'
list_of_lists_sekitar_gedung = kost_raw['sekitar_gedung'].apply(lambda x: clean_sekitar_gedung(x)).tolist()
display(list_of_lists_sekitar_gedung[:5])

# one-hot encode each of record from 'list_of_lists_sekitar_gedung'
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(list_of_lists_sekitar_gedung).transform(list_of_lists_sekitar_gedung)
df_sekitar_gedung = pd.DataFrame(te_ary, columns = te.columns_).replace({True:1, False:0})
display(df_sekitar_gedung.head())

print('total of unique near buildings from the kost:', len(df_sekitar_gedung.columns.tolist()))

[['supermarket',
  'sekolah',
  'rumah sakit',
  'rumah makan',
  'mall',
  'kampus',
  'halte bus'],
 ['supermarket',
  'sekolah',
  'rumah sakit',
  'rumah makan',
  'mall',
  'kampus',
  'halte bus'],
 ['supermarket',
  'sekolah',
  'rumah sakit',
  'rumah makan',
  'mall',
  'kampus',
  'halte bus'],
 ['supermarket',
  'sekolah',
  'rumah sakit',
  'rumah makan',
  'mall',
  'kampus',
  'halte bus'],
 ['supermarket',
  'sekolah',
  'rumah sakit',
  'rumah makan',
  'mall',
  'kampus',
  'halte bus']]

,"-6.16124980680347, 106.8227739846222monas","-6.1626915184816875, 106.84946688291765",3 menit ke perkantoran pemda,5 menit ke ccm,5 menit ke rsud,aeon mall bsd,aeon mall jakarta garden city,aeon mall tanjung barat,agro edukasi wisata ragunan,airport,airzone basketball court,akademi administrasi pembangunan indonesia,akademi bahasa asing nasional,akademi keperawatan pasar rebo,al azhar kelapa gading,al falahiyyah islamic school,al-azhar syifa budi jakarta,alfa mart dan atm ~80m,alfamart,alfamidi,alun alun bandung,alun alun kota bandung,alun-alun kota bandung,ancol,apartemen gading icon,apartemen grand asia afrika,apartemen mediterania gajah mada,apartemen menara kebon jeruk,apartemen parahyangan residence,apartemen taman melati yogyakarta,apotek,apotek dan klinik berkah salamah ciputat,apotek k24,apotek k24 bangka raya,apotek kimia farma,apotek kimia farma bangka raya,apotek kimia farma pasar minggu,apotek kimia farma radio dalam,apotek kimia farma radio dalam premier,apotek roxy poltangan,apotik,apotik medical farma,apotik roxy jatikramat,apotik senopati,arion mall,arsip nasional republik indonesia,ashta district 8,asrama sangkuriang itb,atm,awsy futsal,babakan siliwangi city forest path bandung,badan intelijen negara republik indonesia,badan meteorologi klimatologi dan geofisika indonesia,badan pengawasan keuangan dan pembangunan,badan pengembangan sumber daya manusia industri (bpsdmi) kemenperin,badan siber dan sandi negara (bssn) republik indonesia,balai sidang jakarta convention center,balai sudirman jakarta,bali royal hospital,baltos tamansari,bandar udara atang senjaya,bandar udara halim perdanakusuma,bandar udara internasional adisutjipto,bandar udara internasional soekarno–hatta,bandar udara pondok cabe,bandara halim perdana kusuma,bandara halim perdanakusuma,bandara international soekarno hatta,bandung indah plaza,bandung trade centre mall,bank,bank bca,bank bri corporate university,bank btn kc jakarta cawang,bank dki kantor pusat,barbershop,barito park,basoeki abdullah muesum,batavia futsal,baywalk mall,bca,bca kcp ciputat 2,bca kcp condet,beacon academy,bella terra lifestyle center kelapa gading,bellagio boutique mall,bidakara tower,bina putra arena,bina tunas bangsa school,bintaro jaya xchange mall,bintaro lotte mall,bintaro plaza,bintaro plaza loby utama,bintaro plaza loby utamad,bintaro xchange xxi,binus,binus kampung kijang,binus school serpong,binus syahdan,"binus university, kampus kijang",block m square,blok m,blok m bca,blok m plaza,blok m plaza/square,blok m square,blu plaza,borma dago,bpjs kesehatan cabang jakarta barat (gedung baru),bpjs kesehatan cabang jakarta selatan,bpjs ketenagakerjaan kcp kebayoran lama,bpkp perwakilan provinsi dki jakarta,braga citywalk,branchsto bsd,brawijaya hospital durentiga,brawijaya hospital saharjo,bsd plaza,bsd xtreme park,bsi kcp bandung dago,btc bintaro,btc mall 2,btm bandung trade mall,bumi perkemahan ragunan,bundaran hi,bunderan grand kamala lagoon,busway sarinah,"canisius college, jakarta",carrefour kiaracondong,carro square,cengkareng golf club,central park,central park mall,centro futsal sawah besar pd. pasar jaya,cibubur plaza,cibubur times square,cihampelas walk,cikupa xxi,cilandak marinir driving range,cilandak town square,cilandak town square 3.3km,cipinang indah mall,ciplaz ciledug (ramayana),ciplaz klender (ramayana),ciputra citra towers,ciputra hospital citragarden city,ciputra hospital citraraya,ciputra land,ciputra mall,ciputra world jakarta,citibank tower b,citiwalk sudirman jakarta,citos,citra land,citra land mall,citra raya water world,citra raya world of wonders theme park,citraland waterpark,citywalk sudirman,citywalk sudirman jakarta,ciwalk,ciwalk xxi,coffe shop,coffeshop,como park,d3 keperawatan stikes fatmawati,dago heritage 1917 golf course,damai indah golf,damai indah golf – bsd course,damai indah golf – pik course,danau duta harapan,danau sunter,dekat dengan hartono mall.,dekat dengan kampus – kampus uny.ugm. sanata dharma. upn. amikom. uui.,deka

total of unique near buildings from the kost: 1571


In [15]:
# create new binary column: 'sekitar_gedung_mall' 
results, buildings = aggregate_sekitar_gedung('mal|mall|^bintaro|blok m plaza|blok m square|braga citywalk|btc bintaro|central park|cibubur plaza|town square|cihampelas walk|ciputra world jakarta|citos|citiwalk sudirman jakarta|citra land|citywalk sudirman jakarta|ciwalk|gandaria|plaza|grand indonesia|itc|kota kasablanka|kuningan city|living world alam sutera|lokasari square|lotte mart bintaro|ltc|mangga dua square|margo city|margocity|pamulang square|paskal 23|pejaten village|pik avenue|poins square|pvj bandung|qbig bsd city|revo town bekasi|roxy square|senayan city|senayan park|the breeze bsd city|tm seasons city|trans studio|^wtc')
print(f'{results.sum()} kos-kosan are near "mall"')
# display(buildings)

kost_raw['sekitar_gedung_mall'] = results.replace({False:0, True:1})

2578 kos-kosan are near "mall"


In [16]:
# create new binary column: 'sekitar_gedung_supermarket' 
results, buildings = aggregate_sekitar_gedung('supermarket|super market|superindo|borma dago|carrefour|giant|transmart|harmony swalayan|lotte mart taman surya|lotte mart fatmawati|^mart')
print(f'{results.sum()} kos-kosan are near "supermarket"')
# display(buildings)

kost_raw['sekitar_gedung_supermarket'] = results.replace({False:0, True:1})

1816 kos-kosan are near "supermarket"


In [17]:
# create new binary column: 'sekitar_gedung_kampus' 
results, buildings = aggregate_sekitar_gedung('^kampus|campus|^universitas|univ|university|institut|ipb|^itb|itekes|kuliah|college|faculty|fakultas|perguruan tinggi|politeknik|poltekkes|polytechnic|sae|sekolah tinggi|stan|stikes|stmkg|sttii|^ui|^uin|undiknas|unj|unpad dipatiukur|unpas lengkong|unpat|untar|unud|upbjj-ut jakarta|yayasan bina|yayasan islam raudhatul ulum|unpar|ke itb|akademi')
print(f'{results.sum()} kos-kosan are near "kampus"')
# display(buildings)

kost_raw['sekitar_gedung_kampus'] = results.replace({False:0, True:1})

2354 kos-kosan are near "kampus"


In [18]:
# create new binary column: 'sekitar_gedung_akses kesehatan' 
results, buildings = aggregate_sekitar_gedung('^rs|rumah sakit|hospital|mayapada|mitra keluarga|pusat jantung|apotek|apotik|klinik|laboratorium|puskesmas')
print(f'{results.sum()} kos-kosan are near "healthcare access"')
# display(buildings)

kost_raw['sekitar_gedung_akses kesehatan'] = results.replace({False:0, True:1})

2683 kos-kosan are near "healthcare access"


In [19]:
# create new binary column: 'sekitar_gedung_sekolah' 
results, buildings = aggregate_sekitar_gedung('sekolah$|^tk|^sd|^smp|^sma|^smk|school|sekolah dasar|sekolah menengah|sekolah kristen|sekolah katolik|sekolah islam|catholic school|sampoerna academy bsd|sekolah perancis|academy|yayasan pendidikan permai|𝑺𝑴𝑲 𝑰𝑺𝑳𝑨𝑴 𝑰𝑵𝑺𝑨𝑵 𝑴𝑼𝑳𝑰𝑨|al-azhar syifa budi jakarta')
print(f'{results.sum()} kos-kosan are near "sekolah"')
# display(buildings)

kost_raw['sekitar_gedung_sekolah'] = results.replace({False:0, True:1})

1885 kos-kosan are near "sekolah"


In [20]:
# create new binary column: 'sekitar_gedung_akses transportasi' 
results, buildings = aggregate_sekitar_gedung('bandara|airport|halte|stasiun|mrt|station|terminal')
print(f'{results.sum()} kos-kosan are near "transportation access"')
# display(buildings)

kost_raw['sekitar_gedung_akses transportasi'] = results.replace({False:0, True:1})

2317 kos-kosan are near "transportation access"


## Extract kost's city 

In [21]:
def extract_city(x): 
    alamat_kost_1 = [val.strip().lower() for val in x['alamat_kost_1'].split(',')]
    alamat_kost_1 = ', '.join(alamat_kost_1)
    
    alamat_kost_2 = [val.strip().lower() for val in x['alamat_kost_2'].split(',')]
    alamat_kost_2 = ', '.join(alamat_kost_2)
    
    alamat = alamat_kost_1 + alamat_kost_2
    
    if re.findall('bandung', alamat):
        return 'Bandung'
    
    elif re.findall('yogyakarta|sleman|wirogunan', alamat):
        return 'Yogyakarta'
    
    elif re.findall('bali', alamat):
        return 'Bali'
    
    elif re.findall('jakarta barat', alamat):
        return 'Jakarta Barat'
    
    elif re.findall('jakarta pusat', alamat):
        return 'Jakarta Pusat'
    
    elif re.findall('jakarta selatan', alamat):
        return 'Jakarta Selatan'
    
    elif re.findall('jakarta timur', alamat):
        return 'Jakarta Timur'
    
    elif re.findall('jakarta utara', alamat):
        return 'Jakarta Utara'
    
    elif re.findall('depok', alamat):
        return 'Depok'
    
    elif re.findall('tangerang', alamat):
        return 'Tangerang'
    
    elif re.findall('bekasi', alamat):
        return 'Bekasi'
    
    elif re.findall('bogor', alamat):
        return 'Bogor'
    
    else:
        return 'other'

In [22]:
kost_raw['kota'] = kost_raw.apply(lambda x: extract_city(x), axis = 1)
kost_raw['kota'].value_counts()

Bandung            1076
Jakarta Selatan     653
Jakarta Pusat       329
Jakarta Barat       316
Yogyakarta          311
Bali                226
Tangerang           191
Jakarta Timur       161
Jakarta Utara        93
Depok                35
Bekasi               29
Bogor                10
other                 4
Name: kota, dtype: int64

In [23]:
kost_raw[kost_raw['kota'] == 'other']['alamat_kost_1'].tolist()

['jln. haurpancuh 1 no 132',
 'Jl. Pleburan II No. 4 Semarang Selatan',
 'Jl. Semolowaru Tengah 14 No. 31 Sukolilo Surabaya',
 'Jl. Ngagel Tama Utara 4 No. 17 Kertajaya Gubeng Surabaya']

In [24]:
kost_raw[kost_raw['kota'] == 'other']['alamat_kost_2'].tolist()

['LABUHAN RATU,           KOTA BANDAR LAMPUNG,           JAWA BARAT           40133',
 'PLEBURAN,           SEMARANG SELATAN,           KOTA SEMARANG,           JAWA TENGAH           50241',
 'SEMOLOWARU,           SUKOLILO,           KOTA SURABAYA,           JAWA TIMUR           60119',
 'KERTAJAYA,           SUMUR,           KABUPATEN PANDEGLANG,           DKI JAKARTA']

- `"other"` values in `kota` column contain cities of Semarang, Surabaya, and Bandar Lampung. We will drop this records since the sample are not enough.

In [25]:
# drop "other" values in 'kota' column
kost_raw = kost_raw[kost_raw['kota'] != "other"]
kost_raw.shape

(3430, 45)

## Create new DataFrame i.e. `kost_tipe_kamar`  to store information on `tipe_kamar` columns

We will create a new DataFrame, i.e. `kost_tipe_kamar`, in which each record will describes a room type of each single kost entity. For example, we have a single entity of `kost A` which has two room types, i.e. `room 1` and `room 2`. Therefore the new resulting DataFrame will has 2 records, i.e. `kost A - room 1` and `kost B - room 2`.

`link_kost` will be used as kost IDs since each kost entity in the website will have different link url. In the last section, we will merge/join two DataFrame, i.e. `kost_tipe_kamar` and `kost_raw` using `link_kost` column as the unique key ID.

In [26]:
kost_tipe_kamar = kost_raw[['link_kost', 'tipe_kamar', 'tipe_kamar (nama_kamar)',
                            'tipe_kamar (luas_kamar)', 'tipe_kamar (harga_kamar)',
                            'tipe_kamar (spesifikasi_kamar)', 'tipe_kamar (fasilitas_kamar)']]
kost_tipe_kamar

,link_kost,tipe_kamar,tipe_kamar (nama_kamar),tipe_kamar (luas_kamar),tipe_kamar (harga_kamar),tipe_kamar (spesifikasi_kamar),tipe_kamar (fasilitas_kamar)
0,https://infokost.id/listing/ruoptions-caraka-i...,yes,"[' Pocket Single', ' Pocket Double']","['Luas kamar: 6.5 -- 7.5m', 'Luas kamar: 6.5 -...","[' Rp 1.250.000', ' ', ' Rp 1.250.000', ' ']","[['1 Tempat tidur (100x200) 1 orang', 'Tidak t...","[['AC', 'Lemari', 'TV'], ['AC', 'Lemari', 'TV']]"
1,https://infokost.id/listing/kost-pondok-latifa...,no,[],[],[],[],[]
2,https://infokost.id/listing/kost-the-loft-cobl...,no,[],[],[],[],[]
3,https://infokost.id/listing/kost-perum-alamand...,no,[],[],[],[],[]
4,https://infokost.id/listing/kost-nijisi-house-...,no,[],[],[],[],[]
...,...,...,...,...,...,...,...
3429,https://infokost.id/listing/kost-ibu-sumarni-b...,no,[],[],[],[],[]
3430,https://infokost.id/listing/kost-syifa-sukajad...,no,[],[],[],[],[]
3431,https://infokost.id/listing/kost-griya-narayan...,no,[],[],[],[],[]
3432,https://infokost.id/listing/kost-amira25-cicen...,no,[],[],[],[],[]


In [27]:
# clean the records so they becomes list of lists
def clean_the_records(x):
    temp_list = ast.literal_eval(x)
    x = [val for val in temp_list if val not in [' ']]
    return x

kost_tipe_kamar['tipe_kamar (nama_kamar)'] = kost_tipe_kamar['tipe_kamar (nama_kamar)'].apply(lambda x: clean_the_records(x))
kost_tipe_kamar['tipe_kamar (luas_kamar)'] = kost_tipe_kamar['tipe_kamar (luas_kamar)'].apply(lambda x: clean_the_records(x))
kost_tipe_kamar['tipe_kamar (harga_kamar)'] = kost_tipe_kamar['tipe_kamar (harga_kamar)'].apply(lambda x: clean_the_records(x))
kost_tipe_kamar['tipe_kamar (spesifikasi_kamar)'] = kost_tipe_kamar['tipe_kamar (spesifikasi_kamar)'].apply(lambda x: clean_the_records(x))
kost_tipe_kamar['tipe_kamar (fasilitas_kamar)'] = kost_tipe_kamar['tipe_kamar (fasilitas_kamar)'].apply(lambda x: clean_the_records(x))

In [28]:
%%time

# extract tipe_kamar information
data_tipe_kamar_cleaned = []
for row in range(len(kost_tipe_kamar)):
    if kost_tipe_kamar.iloc[row]['tipe_kamar'] == 'yes':
        for n in range(len(kost_tipe_kamar.iloc[row]['tipe_kamar (nama_kamar)'])):
            temp_dict = {'link_kost':kost_tipe_kamar.iloc[row]['link_kost'],
                         'tipe_kamar':kost_tipe_kamar.iloc[row]['tipe_kamar'],
                         'tipe_kamar (nama_kamar)':kost_tipe_kamar.iloc[row]['tipe_kamar (nama_kamar)'][n],
                         'tipe_kamar (luas_kamar)':kost_tipe_kamar.iloc[row]['tipe_kamar (luas_kamar)'][n],
                         'tipe_kamar (harga_kamar)':kost_tipe_kamar.iloc[row]['tipe_kamar (harga_kamar)'][n],
                         'tipe_kamar (spesifikasi_kamar)':kost_tipe_kamar.iloc[row]['tipe_kamar (spesifikasi_kamar)'][n],
                         'tipe_kamar (fasilitas_kamar)':kost_tipe_kamar.iloc[row]['tipe_kamar (fasilitas_kamar)'][n] 
                        }
            data_tipe_kamar_cleaned.append(temp_dict)
    else:
        temp_dict = {'link_kost':kost_tipe_kamar.iloc[row]['link_kost'],
                     'tipe_kamar':kost_tipe_kamar.iloc[row]['tipe_kamar'],
                     'tipe_kamar (nama_kamar)':kost_tipe_kamar.iloc[row]['tipe_kamar (nama_kamar)'],
                     'tipe_kamar (luas_kamar)':kost_tipe_kamar.iloc[row]['tipe_kamar (luas_kamar)'],
                     'tipe_kamar (harga_kamar)':kost_tipe_kamar.iloc[row]['tipe_kamar (harga_kamar)'],
                     'tipe_kamar (spesifikasi_kamar)':kost_tipe_kamar.iloc[row]['tipe_kamar (spesifikasi_kamar)'],
                     'tipe_kamar (fasilitas_kamar)':kost_tipe_kamar.iloc[row]['tipe_kamar (fasilitas_kamar)'] 
                        }
        data_tipe_kamar_cleaned.append(temp_dict)

Wall time: 2.4 s


In [29]:
kost_tipe_kamar = pd.DataFrame(data_tipe_kamar_cleaned)
kost_tipe_kamar

,link_kost,tipe_kamar,tipe_kamar (nama_kamar),tipe_kamar (luas_kamar),tipe_kamar (harga_kamar),tipe_kamar (spesifikasi_kamar),tipe_kamar (fasilitas_kamar)
0,https://infokost.id/listing/ruoptions-caraka-i...,yes,Pocket Single,Luas kamar: 6.5 -- 7.5m,Rp 1.250.000,"[1 Tempat tidur (100x200) 1 orang, Tidak terma...","[AC, Lemari, TV]"
1,https://infokost.id/listing/ruoptions-caraka-i...,yes,Pocket Double,Luas kamar: 6.5 -- 7.5m,Rp 1.250.000,"[1 Tempat tidur (150x200) 1 orang, Tidak terma...","[AC, Lemari, TV]"
2,https://infokost.id/listing/kost-pondok-latifa...,no,[],[],[],[],[]
3,https://infokost.id/listing/kost-the-loft-cobl...,no,[],[],[],[],[]
4,https://infokost.id/listing/kost-perum-alamand...,no,[],[],[],[],[]
...,...,...,...,...,...,...,...
5019,https://infokost.id/listing/kost-ibu-sumarni-b...,no,[],[],[],[],[]
5020,https://infokost.id/listing/kost-syifa-sukajad...,no,[],[],[],[],[]
5021,https://infokost.id/listing/kost-griya-narayan...,no,[],[],[],[],[]
5022,https://infokost.id/listing/kost-amira25-cicen...,no,[],[],[],[],[]


### Clean `kost_tipe_kamar` DataFrame

#### Clean `tipe_kamar (harga_kamar)` column

In [30]:
# before
display(kost_tipe_kamar['tipe_kamar (harga_kamar)'].head())

# after
kost_tipe_kamar['tipe_kamar (harga_kamar)'] = kost_tipe_kamar['tipe_kamar (harga_kamar)'].str.strip().str.replace('Rp','').str.replace('.','').astype('float').fillna(0)
display(kost_tipe_kamar['tipe_kamar (harga_kamar)'].head())

0     Rp 1.250.000
1     Rp 1.250.000
2               []
3               []
4               []
Name: tipe_kamar (harga_kamar), dtype: object

0    1250000.0
1    1250000.0
2          0.0
3          0.0
4          0.0
Name: tipe_kamar (harga_kamar), dtype: float64

#### Clean `tipe_kamar (luas_kamar)` column

In [31]:
def clean_tipe_kamar_luas_kamar(x):
    x = x.strip('luas kamar:').replace(',','.').replace('-', ' ').replace('tingkat','').strip('()').replace('×','x')
    if x != '':
        if re.findall('x', x):
            if len(x.split('x')) == 2:
                output = x.split('x')
                return float(output[0]) * float(output[1])
            else:
                x = x.split()
                x1 = x[0].split('x')
                x2 = x[1].split('x') 
                x1 = float(x1[0]) * float(x1[1])
                x2 = float(x2[0]) * float(x2[1])
                return (x1 + x2)/2
        else:
            x = x.split()
            if len(x) == 2:
                return (float(x[1]) + float(x[0]))/2
            else:
                return float(x[0])
    else:
        return np.nan

In [32]:
# before
display(kost_tipe_kamar['tipe_kamar (luas_kamar)'].head())

# after
kost_tipe_kamar['tipe_kamar (luas_kamar)'] = kost_tipe_kamar['tipe_kamar (luas_kamar)'].str.lower().str.strip().fillna('0').apply(lambda x: clean_tipe_kamar_luas_kamar(x))
display(kost_tipe_kamar['tipe_kamar (luas_kamar)'].head())

0    Luas kamar: 6.5 -- 7.5m
1    Luas kamar: 6.5 -- 7.5m
2                         []
3                         []
4                         []
Name: tipe_kamar (luas_kamar), dtype: object

0    7.0
1    7.0
2    0.0
3    0.0
4    0.0
Name: tipe_kamar (luas_kamar), dtype: float64

### One-hot encode `kost_tipe_kamar` DataFrame

#### `tipe_kamar (spesifikasi_kamar)`

In [33]:
def clean_spesifikasi_kamar(x):
    x = [val.strip().lower() for val in x]
    return x

def aggregate_spesifikasi_kamar(regex_spesifikasi_kamar):
    selected_columns = [val for val in df_spesifikasi_kamar.columns.tolist() if re.findall(regex_spesifikasi_kamar, val)]
    item_columns = df_spesifikasi_kamar[selected_columns]

    # return category of aggregated items
    return item_columns.sum(axis = 1) >= 1, selected_columns

In [34]:
# create list of lists from 'tipe_kamar (spesifikasi_kamar)'
list_of_lists = kost_tipe_kamar['tipe_kamar (spesifikasi_kamar)'].apply(lambda x: clean_spesifikasi_kamar(x)).tolist()
display(list_of_lists[:5])

# one-hot encode each of record from 'tipe_kamar (spesifikasi_kamar)'
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(list_of_lists).transform(list_of_lists)
df_spesifikasi_kamar = pd.DataFrame(te_ary, columns = te.columns_).replace({True:1, False:0})
display(df_spesifikasi_kamar.head())

print('total of unique room specifications from the kost:', len(df_spesifikasi_kamar.columns.tolist()))

display(df_spesifikasi_kamar.columns.tolist())

[['1 tempat tidur (100x200) 1 orang',
  'tidak termasuk listrik',
  'kamar mandi dalam, shower'],
 ['1 tempat tidur (150x200) 1 orang',
  'tidak termasuk listrik',
  'kamar mandi dalam, shower'],
 [],
 [],
 []]

,0 tempat tidur (0) 1 orang,0 tempat tidur (0) 2 orang,0 tempat tidur (0) 3 orang,0 tempat tidur (100x200) 1 orang,0 tempat tidur (120 x 200) 1 orang,0 tempat tidur (90x200) 1 orang,00 tempat tidur (90x200) 1 orang,00000 tempat tidur (100x200) 1 orang,1 tempat tidur ((90 x 100) - (100 x 200)) 1 orang,1 tempat tidur (100 x 200) 1 orang,1 tempat tidur (100) 2 orang,1 tempat tidur (1000x200) 1 orang,1 tempat tidur (100x180) 1 orang,1 tempat tidur (100x190) 1 orang,1 tempat tidur (100x200),1 tempat tidur (100x200) 1 orang,1 tempat tidur (100x200) 2 orang,1 tempat tidur (110x200) 1 orang,1 tempat tidur (110x200) 2 orang,"1 tempat tidur (12,20) 1 orang",1 tempat tidur (120 x 200 (x2)) 1 orang,1 tempat tidur (120 x 200) 1 orang,1 tempat tidur (120x200 (slide)) 1 orang,1 tempat tidur (120x200),1 tempat tidur (120x200) 1 orang,1 tempat tidur (120x200) 2 orang,"1 tempat tidur (13,50) 1 orang",1 tempat tidur (140 x 200) 1 orang,1 tempat tidur (140x200) 1 orang,1 tempat tidur (150x200) 1 orang,1 tempat tidur (160 x 200) 1 orang,1 tempat tidur (160 x200) 1 orang,1 tempat tidur (160x020) 1 orang,1 tempat tidur (160x200 (x2)),1 tempat tidur (160x200),1 tempat tidur (160x200) 1 orang,1 tempat tidur (160x200) 2 orang,1 tempat tidur (160x200) 3 orang,1 tempat tidur (180 x 200) 1 orang,1 tempat tidur (180x200) 1 orang,1 tempat tidur (180x200) 2 orang,1 tempat tidur (200 x 200) 1 orang,1 tempat tidur (200x200) 1 orang,1 tempat tidur (70x200) 1 orang,"1 tempat tidur (9,50) 1 orang",1 tempat tidur (90 - 100x200) 1 orang,1 tempat tidur (90 x 200) 1 orang,1 tempat tidur (900x200) 1 orang,1 tempat tidur (90x100) 1 orang,1 tempat tidur (90x200 (+90x180 sorong)) 1 orang,1 tempat tidur (90x200 (x2)) 1 orang,1 tempat tidur (90x200 - 100x200) 1 orang,1 tempat tidur (90x200),1 tempat tidur (90x200) 1 orang,1 tempat tidur (90x200) 2 orang,10 tempat tidur (90x200) 1 orang,2 tempat tidur (100 x 200) 1 orang,2 tempat tidur (100 x 200) 2 orang,2 tempat tidur (100x200) 1 orang,2 tempat tidur (100x200) 2 orang,2 tempat tidur (120 x 200) 1 orang,2 tempat tidur (120x200 dan 100x200) 2 orang,2 tempat tidur (120x200) 1 orang,2 tempat tidur (120x200) 2 orang,2 tempat tidur (160x200) 2 orang,2 tempat tidur (160x200) 3 orang,"2 tempat tidur (160x200, 120x200) 1 orang",2 tempat tidur (90 x 200) 1 orang,2 tempat tidur (90x200) 1 orang,2 tempat tidur (90x200) 2 orang,8 tempat tidur (90x200) 1 orang,jendela kearah luar,kamar mandi dalam,"kamar mandi dalam, bathtub, shower, pemanas air, wastafel","kamar mandi dalam, bathtub, shower, wastafel","kamar mandi dalam, kamar mandi luar, shower, wastafel","kamar mandi dalam, pemanas air","kamar mandi dalam, pemanas air, wastafel","kamar mandi dalam, shower","kamar mandi dalam, shower, bathtub, wastafel","kamar mandi dalam, shower, bathtub, wastafel, pemanas air","kamar mandi dalam, shower, pemanas air","kamar mandi dalam, shower, pemanas air, wastafel","kamar mandi dalam, shower, wastafel","kamar mandi dalam, shower, wastafel, pemanas air","kamar mandi dalam, wastafel","kamar mandi dalam, wastafel, pemanas air",kamar mandi luar,"kamar mandi luar, bathtub, shower, pemanas air, wastafel","kamar mandi luar, shower","kamar mandi luar, shower, bathtub, wastafel, pemanas air","kamar mandi luar, shower, pemanas air","kamar mandi luar, shower, pemanas air, wastafel","kamar mandi luar, shower, wastafel","kamar mandi luar, shower, wastafel, pemanas air","kamar mandi luar, wastafel",shower,"shower, pemanas air","shower, wastafel","shower, wastafel, pemanas air",terdapat balkon,termasuk listrik,tidak termasuk listrik
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,

total of unique room specifications from the kost: 103


['0 tempat tidur (0) 1 orang',
 '0 tempat tidur (0) 2 orang',
 '0 tempat tidur (0) 3 orang',
 '0 tempat tidur (100x200) 1 orang',
 '0 tempat tidur (120 x 200) 1 orang',
 '0 tempat tidur (90x200) 1 orang',
 '00 tempat tidur (90x200) 1 orang',
 '00000 tempat tidur (100x200) 1 orang',
 '1 tempat tidur ((90 x 100) - (100 x 200)) 1 orang',
 '1 tempat tidur (100 x 200) 1 orang',
 '1 tempat tidur (100) 2 orang',
 '1 tempat tidur (1000x200) 1 orang',
 '1 tempat tidur (100x180) 1 orang',
 '1 tempat tidur (100x190) 1 orang',
 '1 tempat tidur (100x200)',
 '1 tempat tidur (100x200) 1 orang',
 '1 tempat tidur (100x200) 2 orang',
 '1 tempat tidur (110x200) 1 orang',
 '1 tempat tidur (110x200) 2 orang',
 '1 tempat tidur (12,20) 1 orang',
 '1 tempat tidur (120 x 200 (x2)) 1 orang',
 '1 tempat tidur (120 x 200) 1 orang',
 '1 tempat tidur (120x200 (slide)) 1 orang',
 '1 tempat tidur (120x200)',
 '1 tempat tidur (120x200) 1 orang',
 '1 tempat tidur (120x200) 2 orang',
 '1 tempat tidur (13,50) 1 orang',
 

In [35]:
print(f'Terdapat {kost_tipe_kamar.shape[0]} kamar kos.\n')
print(f'distribusi kost apakah memiliki tipe kamar atau tidak:\n{kost_tipe_kamar.tipe_kamar.value_counts()}')

Terdapat 5024 kamar kos.

distribusi kost apakah memiliki tipe kamar atau tidak:
yes    3506
no     1518
Name: tipe_kamar, dtype: int64


##### inside or outside bathroom

In [36]:
# create new binary column: 'spesifikasi_kamar_KM_dalam' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('kamar mandi dalam|^shower$|^shower, wastafel$')
print(f'{results.sum()} kamar kos memiliki "kamar mandi dalam"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_KM dalam'] = results.replace({False:0, True:1})

2902 kamar kos memiliki "kamar mandi dalam"


In [37]:
# create new binary column: 'spesifikasi_kamar_KM_luar' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('^kamar mandi luar|^shower, wastafel, pemanas air$|^shower, pemanas air')
print(f'{results.sum()} kamar kos memiliki "kamar mandi luar"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_KM luar'] = results.replace({False:0, True:1})

604 kamar kos memiliki "kamar mandi luar"


In [38]:
# refine specific rows according to website
print(kost_tipe_kamar[kost_tipe_kamar['link_kost'] == 'https://infokost.id/listing/kost-murni-3/'].index.values)
kost_tipe_kamar.loc[1850, ['spesifikasi_kamar_KM dalam']] = 0
kost_tipe_kamar.loc[1850, ['spesifikasi_kamar_KM luar']] = 1

print(kost_tipe_kamar[kost_tipe_kamar['link_kost'] == 'https://infokost.id/listing/rukita-little-house-tomang/'].index.values)
kost_tipe_kamar.loc[958, ['spesifikasi_kamar_KM dalam']] = 1
kost_tipe_kamar.loc[958, ['spesifikasi_kamar_KM luar']] = 0

[1850]
[958 959 960 961 962 963 964]


In [39]:
# validate the columns
validate_columns = ['spesifikasi_kamar_KM dalam', 'spesifikasi_kamar_KM luar']

print(f'{kost_tipe_kamar[validate_columns].sum().sum()} total kamar dengan tipe kamar == yes (inside or outside bathroom)')

3506 total kamar dengan tipe kamar == yes (inside or outside bathroom)


##### bathroom specs

In [40]:
# create new binary column: 'spesifikasi_kamar_pemanas air' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('pemanas air')
print(f'{results.sum()} kamar kos memiliki "pemanas air"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_pemanas air'] = results.replace({False:0, True:1})

1827 kamar kos memiliki "pemanas air"


In [41]:
# create new binary column: 'spesifikasi_kamar_shower' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('shower')
print(f'{results.sum()} kamar kos memiliki "shower"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_shower'] = results.replace({False:0, True:1})

2957 kamar kos memiliki "shower"


In [42]:
# create new binary column: 'spesifikasi_kamar_bathtub' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('bathtub')
print(f'{results.sum()} kamar kos memiliki "bathtub"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_bathtub'] = results.replace({False:0, True:1})

37 kamar kos memiliki "bathtub"


In [43]:
# create new binary column: 'spesifikasi_kamar_wastafel' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('wastafel')
print(f'{results.sum()} kamar kos memiliki "wastafel"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_wastafel'] = results.replace({False:0, True:1})

1908 kamar kos memiliki "wastafel"


##### Balcony

In [44]:
# create new binary column: 'spesifikasi_kamar_balkon' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('balkon')
print(f'{results.sum()} kamar kos memiliki "balkon"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_balkon'] = results.replace({False:0, True:1})

176 kamar kos memiliki "balkon"


##### Electricity

In [45]:
# create new binary column: 'spesifikasi_kamar_termasuk listrik' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('^termasuk listrik')
print(f'{results.sum()} kamar kos memiliki "termasuk listrik"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_termasuk listrik'] = results.replace({False:0, True:1})

684 kamar kos memiliki "termasuk listrik"


In [46]:
# create new binary column: 'spesifikasi_kamar_tidak termasuk listrik' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('^tidak termasuk listrik')
print(f'{results.sum()} kamar kos memiliki "tidak termasuk listrik"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_tidak termasuk listrik'] = results.replace({False:0, True:1})

2822 kamar kos memiliki "tidak termasuk listrik"


In [47]:
# validate the columns
validate_columns = ['spesifikasi_kamar_termasuk listrik', 'spesifikasi_kamar_tidak termasuk listrik']

print(f'{kost_tipe_kamar[validate_columns].sum().sum()} total kamar dengan tipe kamar == yes (electricity)' )

3506 total kamar dengan tipe kamar == yes (electricity)


##### Window 

In [48]:
# create new binary column: 'spesifikasi_kamar_jendela kearah luar' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('jendela kearah luar')
print(f'{results.sum()} kamar kos memiliki "jendela kearah luar"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_jendela kearah luar'] = results.replace({False:0, True:1})

2195 kamar kos memiliki "jendela kearah luar"


##### Number of beds

In [49]:
# create new binary column: 'spesifikasi_kamar_0 bed' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('^0 tempat tidur')
print(f'{results.sum()} kamar kos memiliki "0 bed"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_0 bed'] = results.replace({False:0, True:1})

46 kamar kos memiliki "0 bed"


In [50]:
# create new binary column: 'spesifikasi_kamar_1 bed' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('^1 tempat tidur|^10 tempat tidur|^8 tempat tidur|^00000 tempat tidur|^00 tempat tidur')
print(f'{results.sum()} kamar kos memiliki "1 bed"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_1 bed'] = results.replace({False:0, True:1})

3360 kamar kos memiliki "1 bed"


In [51]:
# create new binary column: 'spesifikasi_kamar_2 bed' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('^2 tempat tidur')
print(f'{results.sum()} kamar kos memiliki "2 bed"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_2 bed'] = results.replace({False:0, True:1})

100 kamar kos memiliki "2 bed"


In [52]:
# validate the columns
validate_columns = ['spesifikasi_kamar_0 bed', 'spesifikasi_kamar_1 bed', 'spesifikasi_kamar_2 bed']

print(f'{kost_tipe_kamar[validate_columns].sum().sum()} total kamar dengan tipe kamar == yes (number of beds)' )

3506 total kamar dengan tipe kamar == yes (number of beds)


##### Bed size

In [53]:
# create new binary column: 'spesifikasi_kamar_single bed' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('[(]90x200[)]|[(]90 x 200[)]|90x200 [(]|90 - 100x200|90x200 - 100x200|[(]90 x 100[)] - [(]100 x 200[)]|900x200|90x100|70x200|[(]0[)]|[(]9,50[)]')
print(f'{results.sum()} kamar kos memiliki "single bed"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_single bed'] = results.replace({False:0, True:1})

704 kamar kos memiliki "single bed"


In [54]:
# create new binary column: 'spesifikasi_kamar_long bed' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('[(]100x200[)]|tempat tidur [(]100 x 200[)]|1000x200|100x180|100x190|110x200|[(]100[)]')
print(f'{results.sum()} kamar kos memiliki "long bed"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_long bed'] = results.replace({False:0, True:1})

1054 kamar kos memiliki "long bed"


In [55]:
# create new binary column: 'spesifikasi_kamar_double bed' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('[(]120x200[)]|120 x 200|120x200 dan 100x200|120x200 [(]slide[)]|[(]12,20[)]|[(]13,50[)]|140x200|140 x 200')
print(f'{results.sum()} kamar kos memiliki "double bed"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_double bed'] = results.replace({False:0, True:1})

711 kamar kos memiliki "double bed"


In [56]:
# create new binary column: 'spesifikasi_kamar_queen bed' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('160x200|160 x 200|160x200, 120x200|160x020|150x200|160 x200')
print(f'{results.sum()} kamar kos memiliki "queen bed"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_queen bed'] = results.replace({False:0, True:1})

995 kamar kos memiliki "queen bed"


In [57]:
# create new binary column: 'spesifikasi_kamar_≥king bed' 
results, spesifikasi_kamar = aggregate_spesifikasi_kamar('180 x 200|180x200|200x200|200 x 200')
print(f'{results.sum()} kamar kos memiliki "≥king bed"')
# display(spesifikasi_kamar)

kost_tipe_kamar['spesifikasi_kamar_≥king bed'] = results.replace({False:0, True:1})

42 kamar kos memiliki "≥king bed"


In [58]:
# validate the columns
validate_columns = ['spesifikasi_kamar_single bed', 'spesifikasi_kamar_long bed', 'spesifikasi_kamar_double bed', 'spesifikasi_kamar_queen bed', 'spesifikasi_kamar_≥king bed']

print(f'{kost_tipe_kamar[validate_columns].sum().sum()} total kamar dengan tipe kamar == yes (bed size)' )

3506 total kamar dengan tipe kamar == yes (bed size)


#### `tipe_kamar (fasilitas_kamar)`

In [59]:
def clean_fasilitas_kamar(x):
    x = [val.strip().lower() for val in x]
    return x

In [60]:
# create list of lists from 'tipe_kamar (fasilitas_kamar)'
list_of_lists = kost_tipe_kamar['tipe_kamar (fasilitas_kamar)'].apply(lambda x: clean_fasilitas_kamar(x)).tolist()
display(list_of_lists[:5])

# one-hot encode each of record from 'tipe_kamar (fasilitas_kamar)'
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(list_of_lists).transform(list_of_lists)
df_fasilitas_kamar = pd.DataFrame(te_ary, columns = ['fasilitas_kamar_' + col for col in te.columns_]).replace({True:1, False:0})
display(df_fasilitas_kamar.head())

print('total of unique room facilities from the kost:', len(df_fasilitas_kamar.columns.tolist()))

display(df_fasilitas_kamar.columns.tolist())

[['ac', 'lemari', 'tv'], ['ac', 'lemari', 'tv'], [], [], []]

,fasilitas_kamar_ac,fasilitas_kamar_dispenser,fasilitas_kamar_kipas angin,fasilitas_kamar_kompor,fasilitas_kamar_kulkas,fasilitas_kamar_kursi,fasilitas_kamar_lemari,fasilitas_kamar_meja,fasilitas_kamar_sprei,fasilitas_kamar_tv,fasilitas_kamar_tv kabel
0,1,0,0,0,0,0,1,0,0,1,0
1,1,0,0,0,0,0,1,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0


total of unique room facilities from the kost: 11


['fasilitas_kamar_ac',
 'fasilitas_kamar_dispenser',
 'fasilitas_kamar_kipas angin',
 'fasilitas_kamar_kompor',
 'fasilitas_kamar_kulkas',
 'fasilitas_kamar_kursi',
 'fasilitas_kamar_lemari',
 'fasilitas_kamar_meja',
 'fasilitas_kamar_sprei',
 'fasilitas_kamar_tv',
 'fasilitas_kamar_tv kabel']

In [61]:
# concatenate 'df_fasilitas_kamar' with 'kost_tipe_kamar'
kost_tipe_kamar = pd.concat([kost_tipe_kamar, df_fasilitas_kamar], axis = 1)
print(kost_tipe_kamar.shape)

(5024, 36)


# Merge `kost_raw` and `kost_tipe_kamar`

In [62]:
display(kost_raw.head())
print('kost_raw shape:', kost_raw.shape)

display(kost_tipe_kamar.head())
print('kost_tipe_kamar shape:', kost_tipe_kamar.shape)

,link_kost,nama_kost,alamat_kost_1,alamat_kost_2,latitude,longitude,jenis_kost,luas_kamar,fasilitas_gedung,sekitar_gedung,harga_mulai,tipe_kamar,tipe_kamar (nama_kamar),tipe_kamar (luas_kamar),tipe_kamar (harga_kamar),tipe_kamar (spesifikasi_kamar),tipe_kamar (fasilitas_kamar),fasilitas_gedung_ac,fasilitas_gedung_area makan,fasilitas_gedung_area parkir,fasilitas_gedung_cctv,fasilitas_gedung_cleaning service,fasilitas_gedung_dapur,fasilitas_gedung_dispenser,fasilitas_gedung_gym,fasilitas_gedung_jendela menghadap luar,fasilitas_gedung_kolam renang,fasilitas_gedung_kompor,fasilitas_gedung_kulkas,fasilitas_gedung_laundry,fasilitas_gedung_lift,fasilitas_gedung_meja kompor,fasilitas_gedung_mesin cuci,fasilitas_gedung_microwave,fasilitas_gedung_rooftop,fasilitas_gedung_ruang santai / ruang tamu,fasilitas_gedung_tempat jemur pakaian,fasilitas_gedung_wifi,sekitar_gedung_mall,sekitar_gedung_supermarket,sekitar_gedung_kampus,sekitar_gedung_akses kesehatan,sekitar_gedung_sekolah,sekitar_gedung_akses transportasi,kota
0,https://infokost.id/listing/ruoptions-caraka-i...,Caraka Inn Pesanggrahan,"Jl. Pesanggrahan V No.25, RT.8/RW.7, South Pet...","PETUKANGAN SELATAN, PESANGGRAHAN, ...",-6.2400225,106.754748,Kost Campur,0.0,"[' Area parkir', ' CCTV', ' Ruang santai / rua...","['\xa0\xa0Supermarket', '\xa0\xa0Sekolah', '\x...",1250000.0,yes,"[' Pocket Single', ' Pocket Double']","['Luas kamar: 6.5 -- 7.5m', 'Luas kamar: 6.5 -...","[' Rp 1.250.000', ' ', ' Rp 1.250.000', ' ']","[['1 Tempat tidur (100x200) 1 orang', 'Tidak t...","[['AC', 'Lemari', 'TV'], ['AC', 'Lemari', 'TV']]",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,Jakarta Selatan
1,https://infokost.id/listing/kost-pondok-latifa...,Kost Pondok Latifa Coblong Bandung,jl titimplik no18 rt07 rw09 kelurahan sadang s...,"GEGERKALONG, SUKASARI, KOT...",-6.896961223,107.621984,Kost Putri,9.0,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",700000.0,no,[],[],[],[],[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,Bandung
2,https://infokost.id/listing/kost-the-loft-cobl...,Kost The Loft Coblong Bandung,jln. cisitu baru no. 60b. dago. coblong. bandung,"BATUNUNGGAL, BANDUNG KIDUL, ...",-6.88379416,107.612552,Kost Putra,14.0,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",2100000.0,no,[],[],[],[],[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,Bandung
3,https://infokost.id/listing/kost-perum-alamand...,Kost Perum Alamanda Coblong Bandung,"Jl. Tubagus Ismail Raya No.60, Sekeloa, Kecama...","SADANG SERANG, COBLONG, KO...",-6.885143026,107.622391,Kost Campur,11.0,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1300000.0,no,[],[],[],[],[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,Bandung
4,https://infokost.id/listing/kost-nijisi-house-...,Kost Nijisi House Tipe A Coblong Bandung,"Jl. Kanayakan Baru 23, Dago, Coblong, Bandun","SARIJADI, SUKASARI, KOTA B...",-6.878314919,107.617798,Kost Putri,9.0,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1750000.0,no,[],[],[],[],[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,Bandung


kost_raw shape: (3430, 45)


,link_kost,tipe_kamar,tipe_kamar (nama_kamar),tipe_kamar (luas_kamar),tipe_kamar (harga_kamar),tipe_kamar (spesifikasi_kamar),tipe_kamar (fasilitas_kamar),spesifikasi_kamar_KM dalam,spesifikasi_kamar_KM luar,spesifikasi_kamar_pemanas air,spesifikasi_kamar_shower,spesifikasi_kamar_bathtub,spesifikasi_kamar_wastafel,spesifikasi_kamar_balkon,spesifikasi_kamar_termasuk listrik,spesifikasi_kamar_tidak termasuk listrik,spesifikasi_kamar_jendela kearah luar,spesifikasi_kamar_0 bed,spesifikasi_kamar_1 bed,spesifikasi_kamar_2 bed,spesifikasi_kamar_single bed,spesifikasi_kamar_long bed,spesifikasi_kamar_double bed,spesifikasi_kamar_queen bed,spesifikasi_kamar_≥king bed,fasilitas_kamar_ac,fasilitas_kamar_dispenser,fasilitas_kamar_kipas angin,fasilitas_kamar_kompor,fasilitas_kamar_kulkas,fasilitas_kamar_kursi,fasilitas_kamar_lemari,fasilitas_kamar_meja,fasilitas_kamar_sprei,fasilitas_kamar_tv,fasilitas_kamar_tv kabel
0,https://infokost.id/listing/ruoptions-caraka-i...,yes,Pocket Single,7.0,1250000.0,"[1 Tempat tidur (100x200) 1 orang, Tidak terma...","[AC, Lemari, TV]",1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0
1,https://infokost.id/listing/ruoptions-caraka-i...,yes,Pocket Double,7.0,1250000.0,"[1 Tempat tidur (150x200) 1 orang, Tidak terma...","[AC, Lemari, TV]",1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0
2,https://infokost.id/listing/kost-pondok-latifa...,no,[],0.0,0.0,[],[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,https://infokost.id/listing/kost-the-loft-cobl...,no,[],0.0,0.0,[],[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,https://infokost.id/listing/kost-perum-alamand...,no,[],0.0,0.0,[],[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


kost_tipe_kamar shape: (5024, 36)


In [63]:
# fully merge 'kost_raw[included]' and 'kost_tipe_kamar' using 'link_kost' as the unique IDs
excluded = ['tipe_kamar', 'tipe_kamar (nama_kamar)',
            'tipe_kamar (luas_kamar)', 'tipe_kamar (harga_kamar)',
            'tipe_kamar (spesifikasi_kamar)', 'tipe_kamar (fasilitas_kamar)']
included = [col for col in kost_raw.columns if col not in excluded]

kost_final_data = kost_raw[included].merge(kost_tipe_kamar, 
                                           right_on = 'link_kost', left_on = 'link_kost', how = 'outer')
print('kost_final_data shape:', kost_final_data.shape)

kost_final_data shape: (5024, 74)


## Final checking and cleaning

### Kost's areas

- `0` values in `tipe_kamar (luas_kamar)` column indicate that these values are actually located in `luas_kamar`. While otherwise, `0` values in `luas_kamar` column indicate that these values are actually located in `tipe_kamar (luas_kamar)`.
- `NaN` values in `tipe_kamar (luas_kamar)` indicate that these are actual missing values.

In [64]:
# crosscheck 'luas_kamar' and 'tipe_kamar (luas_kamar)'
luas_kost = kost_final_data[['tipe_kamar', 'luas_kamar', 'tipe_kamar (luas_kamar)']]
display(luas_kost)

# check missing values
print(luas_kost.isnull().sum())

,tipe_kamar,luas_kamar,tipe_kamar (luas_kamar)
0,yes,0.0,7.0
1,yes,0.0,7.0
2,no,9.0,0.0
3,no,14.0,0.0
4,no,11.0,0.0
...,...,...,...
5019,no,9.0,0.0
5020,no,17.5,0.0
5021,no,11.0,0.0
5022,no,17.5,0.0


tipe_kamar                  0
luas_kamar                  0
tipe_kamar (luas_kamar)    24
dtype: int64


In [65]:
# create new column 'luas_kost' as the final room areas of a single room kost
def final_cleaning_luas_kost(x):
    if x['tipe_kamar (luas_kamar)'] == 0.0:
        return x['luas_kamar']
    else:
        return x['tipe_kamar (luas_kamar)']
kost_final_data['luas_kost'] = kost_final_data.apply(lambda x: final_cleaning_luas_kost(x), axis = 1)

# compare
kost_final_data[['tipe_kamar', 'luas_kamar', 'tipe_kamar (luas_kamar)', 'luas_kost']]

,tipe_kamar,luas_kamar,tipe_kamar (luas_kamar),luas_kost
0,yes,0.0,7.0,7.0
1,yes,0.0,7.0,7.0
2,no,9.0,0.0,9.0
3,no,14.0,0.0,14.0
4,no,11.0,0.0,11.0
...,...,...,...,...
5019,no,9.0,0.0,9.0
5020,no,17.5,0.0,17.5
5021,no,11.0,0.0,11.0
5022,no,17.5,0.0,17.5


### Kost's price

- `0` values in `tipe_kamar (harga_kamar)` column indicate that these values are actually located in `harga_mulai`.

In [66]:
# crosscheck 'harga_mulai' and 'tipe_kamar (harga_kamar)'
harga_kost = kost_final_data[['tipe_kamar', 'harga_mulai', 'tipe_kamar (harga_kamar)']]
display(harga_kost)

# check missing values
print(harga_kost.isnull().sum())

,tipe_kamar,harga_mulai,tipe_kamar (harga_kamar)
0,yes,1250000.0,1250000.0
1,yes,1250000.0,1250000.0
2,no,700000.0,0.0
3,no,2100000.0,0.0
4,no,1300000.0,0.0
...,...,...,...
5019,no,900000.0,0.0
5020,no,700000.0,0.0
5021,no,850000.0,0.0
5022,no,850000.0,0.0


tipe_kamar                  0
harga_mulai                 0
tipe_kamar (harga_kamar)    0
dtype: int64


In [67]:
# create a new column, 'harga_kost_per_bulan', as the final price of a single room kost
def final_cleaning_harga_kost(x):
    if x['tipe_kamar (harga_kamar)'] == 0.0:
        return x['harga_mulai']
    else:
        return x['tipe_kamar (harga_kamar)']
kost_final_data['harga_kost_per_bulan'] = kost_final_data.apply(lambda x: final_cleaning_harga_kost(x), axis = 1)

# compare
kost_final_data[['tipe_kamar', 'harga_mulai', 'tipe_kamar (harga_kamar)', 'harga_kost_per_bulan']]

,tipe_kamar,harga_mulai,tipe_kamar (harga_kamar),harga_kost_per_bulan
0,yes,1250000.0,1250000.0,1250000.0
1,yes,1250000.0,1250000.0,1250000.0
2,no,700000.0,0.0,700000.0
3,no,2100000.0,0.0,2100000.0
4,no,1300000.0,0.0,1300000.0
...,...,...,...,...
5019,no,900000.0,0.0,900000.0
5020,no,700000.0,0.0,700000.0
5021,no,850000.0,0.0,850000.0
5022,no,850000.0,0.0,850000.0


### `spesifikasi_kamar_XXX` and `fasilitas_kamar_XXX`

- When `tipe_kamar == 'no'`, it means that we don't have any information from the website page about the room types. So the binary feature values for column names which have prefix `spesifikasi_kamar_XXX` and `fasilitas_kamar_XXX` must be filled with `NaN` instead of `0` values.

In [68]:
# crosscheck binary features values: 'spesifikasi_kamar_XXX' and 'fasilitas_kamar_XXX'
binary_cols = [col for col in kost_final_data.columns if re.findall('^spesifikasi_kamar_', col) or re.findall('^fasilitas_kamar_', col)]
kost_final_data[['tipe_kamar'] + binary_cols]

,tipe_kamar,spesifikasi_kamar_KM dalam,spesifikasi_kamar_KM luar,spesifikasi_kamar_pemanas air,spesifikasi_kamar_shower,spesifikasi_kamar_bathtub,spesifikasi_kamar_wastafel,spesifikasi_kamar_balkon,spesifikasi_kamar_termasuk listrik,spesifikasi_kamar_tidak termasuk listrik,spesifikasi_kamar_jendela kearah luar,spesifikasi_kamar_0 bed,spesifikasi_kamar_1 bed,spesifikasi_kamar_2 bed,spesifikasi_kamar_single bed,spesifikasi_kamar_long bed,spesifikasi_kamar_double bed,spesifikasi_kamar_queen bed,spesifikasi_kamar_≥king bed,fasilitas_kamar_ac,fasilitas_kamar_dispenser,fasilitas_kamar_kipas angin,fasilitas_kamar_kompor,fasilitas_kamar_kulkas,fasilitas_kamar_kursi,fasilitas_kamar_lemari,fasilitas_kamar_meja,fasilitas_kamar_sprei,fasilitas_kamar_tv,fasilitas_kamar_tv kabel
0,yes,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0
1,yes,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0
2,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5019,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5020,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5021,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5022,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
# assign NaN value for each binary feature value which has 'tipe_kamar' == "no"
binary_cols = [col for col in kost_final_data.columns if re.findall('^spesifikasi_kamar_', col) or re.findall('^fasilitas_kamar_', col)]

index = kost_final_data[kost_final_data['tipe_kamar'] == 'no'].index
kost_final_data.loc[index, binary_cols] = np.nan
kost_final_data[['tipe_kamar'] + binary_cols]

,tipe_kamar,spesifikasi_kamar_KM dalam,spesifikasi_kamar_KM luar,spesifikasi_kamar_pemanas air,spesifikasi_kamar_shower,spesifikasi_kamar_bathtub,spesifikasi_kamar_wastafel,spesifikasi_kamar_balkon,spesifikasi_kamar_termasuk listrik,spesifikasi_kamar_tidak termasuk listrik,spesifikasi_kamar_jendela kearah luar,spesifikasi_kamar_0 bed,spesifikasi_kamar_1 bed,spesifikasi_kamar_2 bed,spesifikasi_kamar_single bed,spesifikasi_kamar_long bed,spesifikasi_kamar_double bed,spesifikasi_kamar_queen bed,spesifikasi_kamar_≥king bed,fasilitas_kamar_ac,fasilitas_kamar_dispenser,fasilitas_kamar_kipas angin,fasilitas_kamar_kompor,fasilitas_kamar_kulkas,fasilitas_kamar_kursi,fasilitas_kamar_lemari,fasilitas_kamar_meja,fasilitas_kamar_sprei,fasilitas_kamar_tv,fasilitas_kamar_tv kabel
0,yes,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,yes,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5019,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5020,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5021,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5022,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
print('missing of "tipe_kamar":', sum(kost_final_data['tipe_kamar'] == 'no'))
print(kost_final_data[['tipe_kamar'] + binary_cols].isnull().sum())

missing of "tipe_kamar": 1518
tipe_kamar                                     0
spesifikasi_kamar_KM dalam                  1518
spesifikasi_kamar_KM luar                   1518
spesifikasi_kamar_pemanas air               1518
spesifikasi_kamar_shower                    1518
spesifikasi_kamar_bathtub                   1518
spesifikasi_kamar_wastafel                  1518
spesifikasi_kamar_balkon                    1518
spesifikasi_kamar_termasuk listrik          1518
spesifikasi_kamar_tidak termasuk listrik    1518
spesifikasi_kamar_jendela kearah luar       1518
spesifikasi_kamar_0 bed                     1518
spesifikasi_kamar_1 bed                     1518
spesifikasi_kamar_2 bed                     1518
spesifikasi_kamar_single bed                1518
spesifikasi_kamar_long bed                  1518
spesifikasi_kamar_double bed                1518
spesifikasi_kamar_queen bed                 1518
spesifikasi_kamar_≥king bed                 1518
fasilitas_kamar_ac                     

### `fasilitas_gedung_XXX`

- When `fasilitas_gedung == '[]'`, it means that we don't have any information from the website page about the bulding facility. So the binary feature values for column names with prefix `fasilitas_gedung_XXX` must be filled with `NaN` instead of `0` values.

In [71]:
# crosscheck binary features values: 'fasilitas_gedung_XXX'
binary_cols = [col for col in kost_final_data.columns if re.findall('^fasilitas_gedung_', col)]
kost_final_data[['fasilitas_gedung'] + binary_cols]

,fasilitas_gedung,fasilitas_gedung_ac,fasilitas_gedung_area makan,fasilitas_gedung_area parkir,fasilitas_gedung_cctv,fasilitas_gedung_cleaning service,fasilitas_gedung_dapur,fasilitas_gedung_dispenser,fasilitas_gedung_gym,fasilitas_gedung_jendela menghadap luar,fasilitas_gedung_kolam renang,fasilitas_gedung_kompor,fasilitas_gedung_kulkas,fasilitas_gedung_laundry,fasilitas_gedung_lift,fasilitas_gedung_meja kompor,fasilitas_gedung_mesin cuci,fasilitas_gedung_microwave,fasilitas_gedung_rooftop,fasilitas_gedung_ruang santai / ruang tamu,fasilitas_gedung_tempat jemur pakaian,fasilitas_gedung_wifi
0,"[' Area parkir', ' CCTV', ' Ruang santai / rua...",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,"[' Area parkir', ' CCTV', ' Ruang santai / rua...",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5019,[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5020,[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5021,[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5022,[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
# assign NaN value for each binary feature value which has 'fasilitas_gedung' == "[]"
binary_cols = [col for col in kost_final_data.columns if re.findall('^fasilitas_gedung_', col)]

index = kost_final_data[kost_final_data['fasilitas_gedung'] == '[]'].index
kost_final_data.loc[index, binary_cols] = np.nan
kost_final_data[['fasilitas_gedung'] + binary_cols]

,fasilitas_gedung,fasilitas_gedung_ac,fasilitas_gedung_area makan,fasilitas_gedung_area parkir,fasilitas_gedung_cctv,fasilitas_gedung_cleaning service,fasilitas_gedung_dapur,fasilitas_gedung_dispenser,fasilitas_gedung_gym,fasilitas_gedung_jendela menghadap luar,fasilitas_gedung_kolam renang,fasilitas_gedung_kompor,fasilitas_gedung_kulkas,fasilitas_gedung_laundry,fasilitas_gedung_lift,fasilitas_gedung_meja kompor,fasilitas_gedung_mesin cuci,fasilitas_gedung_microwave,fasilitas_gedung_rooftop,fasilitas_gedung_ruang santai / ruang tamu,fasilitas_gedung_tempat jemur pakaian,fasilitas_gedung_wifi
0,"[' Area parkir', ' CCTV', ' Ruang santai / rua...",0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,"[' Area parkir', ' CCTV', ' Ruang santai / rua...",0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5019,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5020,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5021,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5022,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
print('missing of "fasilitas_gedung":', sum(kost_final_data['fasilitas_gedung'] == '[]'))
print(kost_final_data[['fasilitas_gedung'] + binary_cols].isnull().sum())

missing of "fasilitas_gedung": 1471
fasilitas_gedung                                 0
fasilitas_gedung_ac                           1471
fasilitas_gedung_area makan                   1471
fasilitas_gedung_area parkir                  1471
fasilitas_gedung_cctv                         1471
fasilitas_gedung_cleaning service             1471
fasilitas_gedung_dapur                        1471
fasilitas_gedung_dispenser                    1471
fasilitas_gedung_gym                          1471
fasilitas_gedung_jendela menghadap luar       1471
fasilitas_gedung_kolam renang                 1471
fasilitas_gedung_kompor                       1471
fasilitas_gedung_kulkas                       1471
fasilitas_gedung_laundry                      1471
fasilitas_gedung_lift                         1471
fasilitas_gedung_meja kompor                  1471
fasilitas_gedung_mesin cuci                   1471
fasilitas_gedung_microwave                    1471
fasilitas_gedung_rooftop                      

### `sekitar_gedung_XXX`

- When `sekitar_gedung == '[]'`, it means that we don't have any information from the website page about the nearest building object. So the binary feature values for column names with prefix `sekitar_gedung_XXX` must be filled with `NaN` instead of `0` values.

In [74]:
# crosscheck binary features values: 'sekitar_gedung_XXX'
binary_cols = [col for col in kost_final_data.columns if re.findall('^sekitar_gedung_', col)]
kost_final_data[['sekitar_gedung'] + binary_cols]

,sekitar_gedung,sekitar_gedung_mall,sekitar_gedung_supermarket,sekitar_gedung_kampus,sekitar_gedung_akses kesehatan,sekitar_gedung_sekolah,sekitar_gedung_akses transportasi
0,"['\xa0\xa0Supermarket', '\xa0\xa0Sekolah', '\x...",1,1,1,1,1,1
1,"['\xa0\xa0Supermarket', '\xa0\xa0Sekolah', '\x...",1,1,1,1,1,1
2,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1,1,1,1,1,1
3,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1,1,1,1,1,1
4,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1,1,1,1,1,1
...,...,...,...,...,...,...,...
5019,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1,1,1,1,1,1
5020,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1,1,1,1,1,1
5021,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1,1,1,1,1,1
5022,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1,1,1,1,1,1


In [75]:
# assign NaN value for each binary feature value which has 'sekitar_gedung' == "[]"
binary_cols = [col for col in kost_final_data.columns if re.findall('^sekitar_gedung_', col)]

index = kost_final_data[kost_final_data['sekitar_gedung'] == '[]'].index
kost_final_data.loc[index, binary_cols] = np.nan
kost_final_data[['sekitar_gedung'] + binary_cols]

,sekitar_gedung,sekitar_gedung_mall,sekitar_gedung_supermarket,sekitar_gedung_kampus,sekitar_gedung_akses kesehatan,sekitar_gedung_sekolah,sekitar_gedung_akses transportasi
0,"['\xa0\xa0Supermarket', '\xa0\xa0Sekolah', '\x...",1.0,1.0,1.0,1.0,1.0,1.0
1,"['\xa0\xa0Supermarket', '\xa0\xa0Sekolah', '\x...",1.0,1.0,1.0,1.0,1.0,1.0
2,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1.0,1.0,1.0,1.0,1.0,1.0
3,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1.0,1.0,1.0,1.0,1.0,1.0
4,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
5019,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1.0,1.0,1.0,1.0,1.0,1.0
5020,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1.0,1.0,1.0,1.0,1.0,1.0
5021,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1.0,1.0,1.0,1.0,1.0,1.0
5022,"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",1.0,1.0,1.0,1.0,1.0,1.0


In [76]:
print('missing of "sekitar_gedung":', sum(kost_final_data['sekitar_gedung'] == '[]'))
print(kost_final_data[['sekitar_gedung'] + binary_cols].isnull().sum())

missing of "sekitar_gedung": 377
sekitar_gedung                         0
sekitar_gedung_mall                  377
sekitar_gedung_supermarket           377
sekitar_gedung_kampus                377
sekitar_gedung_akses kesehatan       377
sekitar_gedung_sekolah               377
sekitar_gedung_akses transportasi    377
dtype: int64


## Save the final dataset

In [77]:
kost_final_data.to_csv('dataset/Kost Data (Cleaned).csv', index = False)
kost_final_data

,link_kost,nama_kost,alamat_kost_1,alamat_kost_2,latitude,longitude,jenis_kost,luas_kamar,fasilitas_gedung,sekitar_gedung,harga_mulai,fasilitas_gedung_ac,fasilitas_gedung_area makan,fasilitas_gedung_area parkir,fasilitas_gedung_cctv,fasilitas_gedung_cleaning service,fasilitas_gedung_dapur,fasilitas_gedung_dispenser,fasilitas_gedung_gym,fasilitas_gedung_jendela menghadap luar,fasilitas_gedung_kolam renang,fasilitas_gedung_kompor,fasilitas_gedung_kulkas,fasilitas_gedung_laundry,fasilitas_gedung_lift,fasilitas_gedung_meja kompor,fasilitas_gedung_mesin cuci,fasilitas_gedung_microwave,fasilitas_gedung_rooftop,fasilitas_gedung_ruang santai / ruang tamu,fasilitas_gedung_tempat jemur pakaian,fasilitas_gedung_wifi,sekitar_gedung_mall,sekitar_gedung_supermarket,sekitar_gedung_kampus,sekitar_gedung_akses kesehatan,sekitar_gedung_sekolah,sekitar_gedung_akses transportasi,kota,tipe_kamar,tipe_kamar (nama_kamar),tipe_kamar (luas_kamar),tipe_kamar (harga_kamar),tipe_kamar (spesifikasi_kamar),tipe_kamar (fasilitas_kamar),spesifikasi_kamar_KM dalam,spesifikasi_kamar_KM luar,spesifikasi_kamar_pemanas air,spesifikasi_kamar_shower,spesifikasi_kamar_bathtub,spesifikasi_kamar_wastafel,spesifikasi_kamar_balkon,spesifikasi_kamar_termasuk listrik,spesifikasi_kamar_tidak termasuk listrik,spesifikasi_kamar_jendela kearah luar,spesifikasi_kamar_0 bed,spesifikasi_kamar_1 bed,spesifikasi_kamar_2 bed,spesifikasi_kamar_single bed,spesifikasi_kamar_long bed,spesifikasi_kamar_double bed,spesifikasi_kamar_queen bed,spesifikasi_kamar_≥king bed,fasilitas_kamar_ac,fasilitas_kamar_dispenser,fasilitas_kamar_kipas angin,fasilitas_kamar_kompor,fasilitas_kamar_kulkas,fasilitas_kamar_kursi,fasilitas_kamar_lemari,fasilitas_kamar_meja,fasilitas_kamar_sprei,fasilitas_kamar_tv,fasilitas_kamar_tv kabel,luas_kost,harga_kost_per_bulan
0,https://infokost.id/listing/ruoptions-caraka-i...,Caraka Inn Pesanggrahan,"Jl. Pesanggrahan V No.25, RT.8/RW.7, South Pet...","PETUKANGAN SELATAN, PESANGGRAHAN, ...",-6.2400225,106.754748,Kost Campur,0.0,"[' Area parkir', ' CCTV', ' Ruang santai / rua...","['\xa0\xa0Supermarket', '\xa0\xa0Sekolah', '\x...",1250000.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Jakarta Selatan,yes,Pocket Single,7.0,1250000.0,"[1 Tempat tidur (100x200) 1 orang, Tidak terma...","[AC, Lemari, TV]",1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,7.0,1250000.0
1,https://infokost.id/listing/ruoptions-caraka-i...,Caraka Inn Pesanggrahan,"Jl. Pesanggrahan V No.25, RT.8/RW.7, South Pet...","PETUKANGAN SELATAN, PESANGGRAHAN, ...",-6.2400225,106.754748,Kost Campur,0.0,"[' Area parkir', ' CCTV', ' Ruang santai / rua...","['\xa0\xa0Supermarket', '\xa0\xa0Sekolah', '\x...",1250000.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Jakarta Selatan,yes,Pocket Double,7.0,1250000.0,"[1 Tempat tidur (150x200) 1 orang, Tidak terma...","[AC, Lemari, TV]",1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,7.0,1250000.0
2,https://infokost.id/listing/kost-pondok-latifa...,Kost Pondok Latifa Coblong Bandung,jl titimplik no18 rt07 rw09 kelurahan sadang s...,"GEGERKALONG, SUKASARI, KOT...",-6.896961223,107.621984,Kost Putri,9.0,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",700000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,Bandung,no,[],0.0,0.0,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,700000.0
3,https://infokost.id/listing/kost-the-loft-cobl...,Kost The Loft Coblong Bandung,jln. cisitu baru no. 60b. dago. coblong. bandung,"BATUNUNGGAL, BANDUNG KIDUL, ...",-6.88379416,107.612552,Kost Putra,14.0,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",2100000.0,NaN,NaN,NaN,NaN,NaN,NaN,